# ML4CREST: A Watering Subsystem with Machine Learning Calibration

In [1]:
# so we don't have to reload the kernel everytime we change something. This is useful for development. It can be removed at deploy-time, but doesn't harm anybody.
%load_ext autoreload
%autoreload 2

# use most of the browser size, don't do only the narrow thing in the middle
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# avoid being flooded by log messages. Warnings are enough (let's hope we don't have any)
import logging
logging.basicConfig(level=logging.WARNING)  
log = logging.getLogger("src.simulator.conditiontimedchangecalculator")
# log.setLevel(logging.DEBUG)
z3log = logging.getLogger("src.simulator.z3calculator")
# z3log.setLevel(logging.DEBUG)

In [2]:
# import all CREST model concepts
from src.model import *

# import the CREST simulator
from src.simulator.simulator import Simulator

# import a library that can visualise our systems (using ELKjs)
from src.ui import elk

In [3]:
# define the resources within our system. We do it as static objects in a class so we have them all bundled together.
class Resources(object):
    on_off = Resource("switch", ["on", "off"])
    flow = Resource("L/min", REAL)
    water = Resource("Liters", REAL)

In [4]:
class Pump(Entity):
    """
    A pump that is placed in an assumingly infinite source of water, such as a very big water tank, lake or similar. 
    We therefore don't model that the pump's inflow can vary. In fact the pump only has an outflow.
    
    For this example we also disregard the electricity that is required for the pump to operate, as we're only interested in modelling the water and spillage.
    """
    switch = Input(resource=Resources.on_off, value="off")
    output = Output(resource=Resources.flow, value=0)
    
    """ States """
    on = State()
    off = current = State()
    
    """ Transitions """
    off_to_on = Transition(source=off, target=on, guard=(lambda self: self.switch.value == "on"))
    on_to_off = Transition(source=on, target=off, guard=(lambda self: self.switch.value == "off"))
    
    """ Updates """
    @update(state=on, target=output)
    def set_on(self, dt=0):
        return 0.025127

    @update(state=off, target=output)
    def set_off(self, dt=0):
        return 0

elk.plot(Pump())

In [5]:
class Tank(Entity):
    """
    A tank is a very simple system. It has one inflow and one outflow. Inflow is defined by the water source, i.e. it's an input. 
    Outflow is defined by the tank itself. In our case it depends on size of the hose leaving the tank (static) and the current volume (more volume, more output due to pressure).
    We assume the outflow is piecewise linear, for simplicity. We could also try something more complex, but that's overkill...
    
    Additionally to inflow and outflow we will also measure how much water left the tank in total (quantity) 
    and how much water was spilt because we filled the volume higher than the capacity (spill).
    """
    inflow = Input(resource=Resources.flow, value=0.025127)  # the current inflow
    outflow = Output(resource=Resources.flow, value=0)  # the current outflow
    spill = Output(resource=Resources.water, value=0)  # the amount of water spilt
    quantity = Output(resource=Resources.water, value=0)  # the volume of water that left the tank
    
    maxoutflow = Local(resource=Resources.flow, value=0.001941) # the maximum outflow
    volume = Local(resource=Resources.water, value=0)  # the currently held water
    capacity = Local(resource=Resources.water, value=2)  # the maximum capacity, if we fill more, the rest is spilt
        
    """ States """
    full = State()
    partially = current = State()
    empty = State()
        
    """ Transitions """
    @transition(source=partially, target=full)
    def to_full(self):
        return self.volume.value >= self.capacity.value
    
    @transition(source=partially, target=empty)
    def to_empty(self):
        return self.volume.value <= 0
    
    @transition(source=[empty,full], target=partially)
    def to_partially(self):
        return 0 < self.volume.value and self.volume.value < self.capacity.value
    
    """ Updates """
    @update(state=[empty,partially,full], target=volume)
    def set_volume(self, dt):
        netflow = self.inflow.value - self.outflow.value
        theoretical = self.volume.value + netflow * dt
        if theoretical >= self.capacity.value:
            return self.capacity.value
        else:
            return theoretical
        
    @update(state=[partially,full], target=outflow)
    def set_outflow_nonempty(self, dt):
        if self.volume.pre > 0.349:
            return 0.001941
        else:
            return 0.001323
    
    @update(state=empty, target=outflow)
    def set_outflow_empty(self, dt):
        if self.inflow.value > self.maxoutflow.value:
            return self.maxoutflow.value
        else:
            return self.inflow.value
        
    @update(state=[empty,partially,full], target=quantity)
    def increase_quantity(self, dt):
        return self.quantity.value + self.outflow.value * dt
    
    @update(state=full, target=spill)
    def increase_spill(self, dt):
        netflow = self.inflow.value - self.outflow.value
        if netflow > 0:
            return self.spill.value + netflow * dt
        else:
            return self.spill.value
    
elk.plot(Tank())    

In [10]:
class WateringSubsystem(Entity):
    """
    The system is very simple. One pump, one tank. Hook up the connections.
    We connect the system's switch-input to the pump, the pump to the tank, and the tank's outputs to the system's outputs.
    Done. Check out the plot below for more info.
    """
    
    """ Ports """
    switch = Input(resource=Resources.on_off, value="off")  # turn the pump on / off
    output = Output(resource=Resources.flow, value=0)  # the current outflow of the system
    spill = Output(resource=Resources.water, value=0)  # the amount of water spilt
    quantity = Output(resource=Resources.water, value=0)  # the water that was poured over he plants
    
    """ Subentities """
    pump = Pump()
    tank = Tank()
    
    """ State (no transitions) """
    state = current = State()
    
    """ Influences """
    connect_switch = Influence(source=switch, target=pump.switch)
    connect_pump_to_tank = Influence(source=pump.output, target=tank.inflow)
    connect_output = Influence(source=tank.outflow, target=output)
    connect_spill = Influence(source=tank.spill, target=spill)
    connect_quantity = Influence(source=tank.quantity, target=quantity)
    
    def __init__(self, inflow_function=None, outflow_function=None):
        pass
    
sim = Simulator(WateringSubsystem())
sim.stabilise()

"""
This is the list of simulation phases. 
We define the value of the switch to set, and the time to advance with that setting.
We iterate over this list and then we're done. 
"""
evolution = [
    ("on", 5),
    ("off", 30),
    ("on", 5),
    ("off", 30),
    ("on", 5),
    ("off", 30)
]

# advance according to evolution
for p in evolution:
    sim.system.switch.value = p[0]
    sim.advance(p[1])
    
# plot the system
sim.plot()

In [11]:
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)  # don't talk to the plotly server, plot locally within a jupyter notebook


def plot_ports(sim, ytitle, ports):
    """This function takes a simulator and a list of ports, and plots the traces of these ports."""
    store = sim.traces.datastore  # get the trace-datastore from the simulator; the datastore is a mapping that stores ports and their time-value traces and entities and their time-current states traces
    
    # create the lines we want to plot
    lines = []

    # create port traces
    for port in ports:
        lines.append(
            go.Scatter(x=[t[0] for t in store[port]], y=[t[1] for t in store[port]], 
                   name=f"{port._name} ({port.resource.unit})", # append the unit to the name in the legend
                   mode="markers+lines", marker={'symbol': 'x', 'size': 5}, line={'width': 1}  
                )
        )
        
    # create a figure object
    fig = go.Figure(
        data=lines,
        layout=go.Layout(
            showlegend=True, 
            xaxis=dict(title='time (seconds)'),
            yaxis=dict(title=ytitle)
        )
    )
    
    # plot it!
    plotly.offline.iplot(fig)

def plot_sim(sim):
    """
    Creates three plots:
        - Switch state over time
        - Flow measures over time
        - Volume measures over time
    """
    plot_ports(sim, "switch (on/off)", [sim.system.switch])
    plot_ports(sim, "flow (litres/sec)", [sim.system.tank.inflow, sim.system.tank.outflow])
    plot_ports(sim, "volume (litres)", [sim.system.tank.volume, sim.system.tank.spill, sim.system.tank.quantity, sim.system.tank.capacity])
    

plot_sim(sim)